In [ ]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')
# os.chdir("drive/MyDrive/Bootcamp_Henry/pro_final")
# print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/Bootcamp_Henry/pro_final


# **Objetivo del Cuaderno Jupyter**
- Creación del Maestro de Categorías de Restaurantes en Yelp.
    - Tomar en cuenta el archivo `categorias_yelp.xlsx` que es donde se vé la selección y asignación de manera manual de las categorías que pertenecen al sector de Restaurantes

In [ ]:
import pandas as pd

In [ ]:
categorias_viejas_nuevas = pd.read_excel("datasets/categorias_yelp.xlsx", sheet_name="ordenado")

business = pd.read_pickle('datasets/yelp/archivos_originales/business.pkl')
business = business.iloc[:,0:14]

In [ ]:
print(f'Total Categorias Originales Pertenecientes a Restautantes: {categorias_viejas_nuevas["categoria_original"].unique().shape[0]}')
print(f'Total Categorias Actuales Pertenecientes a Restautantes: {categorias_viejas_nuevas["nueva_categoria"].unique().shape[0]}')

Total Categorias Originales Pertenecientes a Restautantes: 194
Total Categorias Actuales Pertenecientes a Restautantes: 14


Ahora vamos a interactuar con las Categorias del Maestro de Categorias del archivo de excel.. y Vamos a Crear una Tabla `y_categories_business` mapeando las categorias nuevas a las viejas juntos con los `business_id`

In [ ]:
y_categories_business = business[['business_id', 'categories']]

y_categories_business['lista_categorias'] = y_categories_business['categories'].str.split(', ')

y_categories_business = y_categories_business.drop(columns=['categories'])

y_categories_business = y_categories_business.explode('lista_categorias')
y_categories_business = y_categories_business.reset_index(drop=True)

y_categories_business = y_categories_business.rename(columns={'lista_categorias': 'category'})

y_categories_business['category'] = y_categories_business['category'].str.lower().str.strip()

y_categories_business['category_general'] = ''

In [ ]:
y_categories_business.head()

,business_id,category,category_general
0,Pns2l4eNsfO8kk83dixA6A,doctors,
1,Pns2l4eNsfO8kk83dixA6A,traditional chinese medicine,
2,Pns2l4eNsfO8kk83dixA6A,naturopathic/holistic,
3,Pns2l4eNsfO8kk83dixA6A,acupuncture,
4,Pns2l4eNsfO8kk83dixA6A,health & medical,


In [ ]:
lista_categorias_originales_comida = categorias_viejas_nuevas['categoria_original'].unique().tolist()

Vamos a Mapear con un ciclo for... Los registros que queden con `category_general` igual a " " se tratan de categorías que no pertenecen al sector en estudio

In [ ]:
"""
Columna 1 --> category
Columna 2 --> category_general
"""
for i in range(y_categories_business.shape[0]):
  if y_categories_business.iloc[i,1] in lista_categorias_originales_comida:
    mask = categorias_viejas_nuevas['categoria_original'] == y_categories_business.iloc[i,1]
    y_categories_business.iloc[i,2] = categorias_viejas_nuevas.loc[mask]['nueva_categoria'].values[0]

print(y_categories_business.shape)
y_categories_business.head()

(668695, 3)


,business_id,category,category_general
0,Pns2l4eNsfO8kk83dixA6A,doctors,
1,Pns2l4eNsfO8kk83dixA6A,traditional chinese medicine,
2,Pns2l4eNsfO8kk83dixA6A,naturopathic/holistic,
3,Pns2l4eNsfO8kk83dixA6A,acupuncture,
4,Pns2l4eNsfO8kk83dixA6A,health & medical,


In [ ]:
mask = y_categories_business['category_general'] == ''
y_categories_business = y_categories_business.loc[~mask]

print(y_categories_business.shape)
y_categories_business.head()

(223831, 3)


,business_id,category,category_general
17,MTSW4McQd7CbVtyjqoe9mw,restaurants,RESTAURANTS
18,MTSW4McQd7CbVtyjqoe9mw,food,RESTAURANTS
19,MTSW4McQd7CbVtyjqoe9mw,bubble tea,CAFETERIAS - PANADERIAS - COMIDAS LIGERAS
20,MTSW4McQd7CbVtyjqoe9mw,coffee & tea,CAFETERIAS - PANADERIAS - COMIDAS LIGERAS
21,MTSW4McQd7CbVtyjqoe9mw,bakeries,CAFETERIAS - PANADERIAS - COMIDAS LIGERAS


Guardando la tabla

In [ ]:
y_categories_business.to_parquet('datasets/categorias/y_categories_business.parquet')